In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
loans_df = pd.read_csv('loan_status.csv')
loans_df.head()

,Loan_Status,Current_Loan_Amount,Term,Credit_Score,Annual_Income,Years_in_current_job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens
0,Fully_Paid,99999999,Short_Term,741.0,2231892.0,8_years,Own_Home,Debt_Consolidation,29200.53,14.9,29.0,18,1,297996,750090.0,0.0,0.0
1,Fully_Paid,217646,Short_Term,730.0,1184194.0,<_1_year,Home_Mortgage,Debt_Consolidation,10855.08,19.6,10.0,13,1,122170,272052.0,1.0,0.0
2,Fully_Paid,548746,Short_Term,678.0,2559110.0,2_years,Rent,Debt_Consolidation,18660.28,22.6,33.0,4,0,437171,555038.0,0.0,0.0
3,Fully_Paid,99999999,Short_Term,728.0,714628.0,3_years,Rent,Debt_Consolidation,11851.06,16.0,76.0,16,0,203965,289784.0,0.0,0.0
4,Fully_Paid,99999999,Short_Term,740.0,776188.0,<_1_year,Own_Home,Debt_Consolidation,11578.22,8.5,25.0,6,0,134083,220220.0,0.0,0.0


In [2]:
# Generate our categorical variable list
loans_cat = loans_df.dtypes[loans_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
loans_df[loans_cat].nunique()

Loan_Status              2
Term                     2
Years_in_current_job    11
Home_Ownership           4
Purpose                  7
dtype: int64

In [3]:
# Check the unique value counts to see if binning is required
loans_df.Years_in_current_job.value_counts()

10+_years    13149
2_years       3225
3_years       2997
<_1_year      2699
5_years       2487
4_years       2286
1_year        2247
6_years       2109
7_years       2082
8_years       1675
9_years       1467
Name: Years_in_current_job, dtype: int64

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(loans_df[loans_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(loans_cat)
encode_df.head()

,Loan_Status_Fully_Paid,Loan_Status_Not_Paid,Term_Long_Term,Term_Short_Term,Years_in_current_job_10+_years,Years_in_current_job_1_year,Years_in_current_job_2_years,Years_in_current_job_3_years,Years_in_current_job_4_years,Years_in_current_job_5_years,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
# Merge one-hot encoded features and drop the originals
loans_df = loans_df.merge(encode_df,left_index=True, right_index=True)
loans_df = loans_df.drop(loans_cat,1)
loans_df.head()

,Current_Loan_Amount,Credit_Score,Annual_Income,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,99999999,741.0,2231892.0,29200.53,14.9,29.0,18,1,297996,750090.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,217646,730.0,1184194.0,10855.08,19.6,10.0,13,1,122170,272052.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,548746,678.0,2559110.0,18660.28,22.6,33.0,4,0,437171,555038.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,99999999,728.0,714628.0,11851.06,16.0,76.0,16,0,203965,289784.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,99999999,740.0,776188.0,11578.22,8.5,25.0,6,0,134083,220220.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [6]:
# Remove loan status target from features data
y = loans_df.Loan_Status_Fully_Paid
X = loans_df.drop(columns=["Loan_Status_Fully_Paid","Loan_Status_Not_Paid"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.849


In [8]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
854/854 [==============================] - 1s 599us/step - loss: 0.4556 - accuracy: 0.8232
Epoch 2/50
854/854 [==============================] - 1s 615us/step - loss: 0.3870 - accuracy: 0.8499
Epoch 3/50
854/854 [==============================] - 1s 614us/step - loss: 0.3893 - accuracy: 0.8452
Epoch 4/50
854/854 [==============================] - 0s 576us/step - loss: 0.3824 - accuracy: 0.8475
Epoch 5/50
854/854 [==============================] - 1s 696us/step - loss: 0.3800 - accuracy: 0.8481
Epoch 6/50
854/854 [==============================] - 1s 610us/step - loss: 0.3820 - accuracy: 0.8468
Epoch 7/50
854/854 [==============================] - 1s 599us/step - loss: 0.3779 - accuracy: 0.8493
Epoch 8/50
854/854 [==============================] - 1s 691us/step - loss: 0.3672 - accuracy: 0.8549
Epoch 9/50
854/854 [==============================] - 1s 691us/step - loss: 0.3711 - accuracy: 0.8521
Epoch 10/50
854/854 [==============================] - 1s 783us/step - loss: 0.369

In [9]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [10]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
 69/854 [=>............................] - ETA: 0s - loss: 0.3791 - accuracy: 0.8388  
Epoch 00001: saving model to checkpoints\weights.01.hdf5
120/854 [===>..........................] - ETA: 0s - loss: 0.3675 - accuracy: 0.8461
Epoch 00001: saving model to checkpoints\weights.01.hdf5
281/854 [========>.....................] - ETA: 0s - loss: 0.3595 - accuracy: 0.8506
Epoch 00001: saving model to checkpoints\weights.01.hdf5
343/854 [===========>..................] - ETA: 0s - loss: 0.3601 - accuracy: 0.8503
Epoch 00001: saving model to checkpoints\weights.01.hdf5
487/854 [================>.............] - ETA: 0s - loss: 0.3612 - accuracy: 0.8499
Epoch 00001: saving model to checkpoints\weights.01.hdf5
551/854 [==================>...........] - ETA: 0s - loss: 0.3615 - accuracy: 0.8498
Epoch 00001: saving model to checkpoints\weights.01.hdf5
670/854 [======================>.......] - ETA: 0s - loss: 0.3615 - accuracy: 0.8500
Epoch 00001: saving model to checkpoints\weights.

206/854 [======>.......................] - ETA: 0s - loss: 0.3675 - accuracy: 0.8497
Epoch 00007: saving model to checkpoints\weights.07.hdf5
359/854 [===========>..................] - ETA: 0s - loss: 0.3671 - accuracy: 0.8489
Epoch 00007: saving model to checkpoints\weights.07.hdf5
420/854 [=============>................] - ETA: 0s - loss: 0.3664 - accuracy: 0.8490
Epoch 00007: saving model to checkpoints\weights.07.hdf5
564/854 [==================>...........] - ETA: 0s - loss: 0.3655 - accuracy: 0.8494
Epoch 00007: saving model to checkpoints\weights.07.hdf5
604/854 [====================>.........] - ETA: 0s - loss: 0.3652 - accuracy: 0.8495
Epoch 00007: saving model to checkpoints\weights.07.hdf5
767/854 [=========================>....] - ETA: 0s - loss: 0.3643 - accuracy: 0.8500
Epoch 00007: saving model to checkpoints\weights.07.hdf5
854/854 [==============================] - 1s 752us/step - loss: 0.3639 - accuracy: 0.8503
Epoch 8/100
  1/854 [..............................] - ET

404/854 [=============>................] - ETA: 0s - loss: 0.3485 - accuracy: 0.8604
Epoch 00013: saving model to checkpoints\weights.13.hdf5
471/854 [===============>..............] - ETA: 0s - loss: 0.3498 - accuracy: 0.8596
Epoch 00013: saving model to checkpoints\weights.13.hdf5
641/854 [=====================>........] - ETA: 0s - loss: 0.3519 - accuracy: 0.8583
Epoch 00013: saving model to checkpoints\weights.13.hdf5
703/854 [=======================>......] - ETA: 0s - loss: 0.3526 - accuracy: 0.8579
Epoch 00013: saving model to checkpoints\weights.13.hdf5
768/854 [=========================>....] - ETA: 0s - loss: 0.3531 - accuracy: 0.8576
Epoch 00013: saving model to checkpoints\weights.13.hdf5
854/854 [==============================] - 1s 742us/step - loss: 0.3537 - accuracy: 0.8572
Epoch 14/100
 76/854 [=>............................] - ETA: 0s - loss: 0.3285 - accuracy: 0.8716
Epoch 00014: saving model to checkpoints\weights.14.hdf5
142/854 [===>..........................] - E

558/854 [==================>...........] - ETA: 0s - loss: 0.3572 - accuracy: 0.8541
Epoch 00019: saving model to checkpoints\weights.19.hdf5
713/854 [========================>.....] - ETA: 0s - loss: 0.3567 - accuracy: 0.8541
Epoch 00019: saving model to checkpoints\weights.19.hdf5
770/854 [==========================>...] - ETA: 0s - loss: 0.3567 - accuracy: 0.8541
Epoch 00019: saving model to checkpoints\weights.19.hdf5
854/854 [==============================] - 1s 828us/step - loss: 0.3568 - accuracy: 0.8539
Epoch 20/100
  1/854 [..............................] - ETA: 0s - loss: 0.3248 - accuracy: 0.8750
Epoch 00020: saving model to checkpoints\weights.20.hdf5
162/854 [====>.........................] - ETA: 0s - loss: 0.3552 - accuracy: 0.8528
Epoch 00020: saving model to checkpoints\weights.20.hdf5
232/854 [=======>......................] - ETA: 0s - loss: 0.3571 - accuracy: 0.8510
Epoch 00020: saving model to checkpoints\weights.20.hdf5
286/854 [=========>....................] - E

790/854 [==========================>...] - ETA: 0s - loss: 0.3514 - accuracy: 0.8564
Epoch 00025: saving model to checkpoints\weights.25.hdf5
854/854 [==============================] - 1s 799us/step - loss: 0.3519 - accuracy: 0.8561
Epoch 26/100
  1/854 [..............................] - ETA: 0s - loss: 0.3254 - accuracy: 0.8438
Epoch 00026: saving model to checkpoints\weights.26.hdf5
135/854 [===>..........................] - ETA: 0s - loss: 0.3586 - accuracy: 0.8514
Epoch 00026: saving model to checkpoints\weights.26.hdf5
193/854 [=====>........................] - ETA: 0s - loss: 0.3591 - accuracy: 0.8511
Epoch 00026: saving model to checkpoints\weights.26.hdf5
339/854 [==========>...................] - ETA: 0s - loss: 0.3573 - accuracy: 0.8520
Epoch 00026: saving model to checkpoints\weights.26.hdf5
395/854 [============>.................] - ETA: 0s - loss: 0.3571 - accuracy: 0.8522
Epoch 00026: saving model to checkpoints\weights.26.hdf5
534/854 [=================>............] - E

...

270/854 [========>.....................] - ETA: 0s - loss: 0.3554 - accuracy: 0.8488
Epoch 00075: saving model to checkpoints\weights.75.hdf5
400/854 [=============>................] - ETA: 0s - loss: 0.3539 - accuracy: 0.8509
Epoch 00075: saving model to checkpoints\weights.75.hdf5
464/854 [===============>..............] - ETA: 0s - loss: 0.3536 - accuracy: 0.8515
Epoch 00075: saving model to checkpoints\weights.75.hdf5
524/854 [=================>............] - ETA: 0s - loss: 0.3532 - accuracy: 0.8521
Epoch 00075: saving model to checkpoints\weights.75.hdf5
682/854 [======================>.......] - ETA: 0s - loss: 0.3524 - accuracy: 0.8533
Epoch 00075: saving model to checkpoints\weights.75.hdf5
721/854 [========================>.....] - ETA: 0s - loss: 0.3523 - accuracy: 0.8535
Epoch 00075: saving model to checkpoints\weights.75.hdf5
854/854 [==============================] - 1s 847us/step - loss: 0.3522 - accuracy: 0.8540
Epoch 76/100
  1/854 [..............................] - E

421/854 [=============>................] - ETA: 0s - loss: 0.3442 - accuracy: 0.8625
Epoch 00081: saving model to checkpoints\weights.81.hdf5
570/854 [===================>..........] - ETA: 0s - loss: 0.3456 - accuracy: 0.8611
Epoch 00081: saving model to checkpoints\weights.81.hdf5
628/854 [=====================>........] - ETA: 0s - loss: 0.3460 - accuracy: 0.8608
Epoch 00081: saving model to checkpoints\weights.81.hdf5
770/854 [==========================>...] - ETA: 0s - loss: 0.3466 - accuracy: 0.8601
Epoch 00081: saving model to checkpoints\weights.81.hdf5
854/854 [==============================] - 1s 746us/step - loss: 0.3470 - accuracy: 0.8598
Epoch 82/100
  1/854 [..............................] - ETA: 0s - loss: 0.4107 - accuracy: 0.8125
Epoch 00082: saving model to checkpoints\weights.82.hdf5
 45/854 [>.............................] - ETA: 0s - loss: 0.3692 - accuracy: 0.8531
Epoch 00082: saving model to checkpoints\weights.82.hdf5
216/854 [======>.......................] - E

599/854 [====================>.........] - ETA: 0s - loss: 0.3528 - accuracy: 0.8548
Epoch 00087: saving model to checkpoints\weights.87.hdf5
736/854 [========================>.....] - ETA: 0s - loss: 0.3528 - accuracy: 0.8548
Epoch 00087: saving model to checkpoints\weights.87.hdf5
854/854 [==============================] - 1s 766us/step - loss: 0.3526 - accuracy: 0.8550
Epoch 88/100
  1/854 [..............................] - ETA: 0s - loss: 0.3941 - accuracy: 0.8438
Epoch 00088: saving model to checkpoints\weights.88.hdf5
 95/854 [==>...........................] - ETA: 0s - loss: 0.3371 - accuracy: 0.8646
Epoch 00088: saving model to checkpoints\weights.88.hdf5
133/854 [===>..........................] - ETA: 0s - loss: 0.3352 - accuracy: 0.8653
Epoch 00088: saving model to checkpoints\weights.88.hdf5
284/854 [========>.....................] - ETA: 0s - loss: 0.3401 - accuracy: 0.8613
Epoch 00088: saving model to checkpoints\weights.88.hdf5
398/854 [============>.................] - E

819/854 [===========================>..] - ETA: 0s - loss: 0.3513 - accuracy: 0.8557
Epoch 00093: saving model to checkpoints\weights.93.hdf5
854/854 [==============================] - 1s 997us/step - loss: 0.3513 - accuracy: 0.8558
Epoch 94/100
  1/854 [..............................] - ETA: 0s - loss: 0.1159 - accuracy: 1.0000
Epoch 00094: saving model to checkpoints\weights.94.hdf5
170/854 [====>.........................] - ETA: 0s - loss: 0.3233 - accuracy: 0.8682
Epoch 00094: saving model to checkpoints\weights.94.hdf5
239/854 [=======>......................] - ETA: 0s - loss: 0.3285 - accuracy: 0.8662
Epoch 00094: saving model to checkpoints\weights.94.hdf5
368/854 [===========>..................] - ETA: 0s - loss: 0.3341 - accuracy: 0.8638
Epoch 00094: saving model to checkpoints\weights.94.hdf5
464/854 [===============>..............] - ETA: 0s - loss: 0.3367 - accuracy: 0.8627
Epoch 00094: saving model to checkpoints\weights.94.hdf5
523/854 [=================>............] - E

  1/854 [..............................] - ETA: 0s - loss: 0.2242 - accuracy: 0.9375
Epoch 00100: saving model to checkpoints\weights.100.hdf5
 71/854 [=>............................] - ETA: 0s - loss: 0.3468 - accuracy: 0.8590
Epoch 00100: saving model to checkpoints\weights.100.hdf5
241/854 [=======>......................] - ETA: 0s - loss: 0.3490 - accuracy: 0.8556
Epoch 00100: saving model to checkpoints\weights.100.hdf5
296/854 [=========>....................] - ETA: 0s - loss: 0.3493 - accuracy: 0.8556
Epoch 00100: saving model to checkpoints\weights.100.hdf5
444/854 [==============>...............] - ETA: 0s - loss: 0.3490 - accuracy: 0.8559
Epoch 00100: saving model to checkpoints\weights.100.hdf5
499/854 [================>.............] - ETA: 0s - loss: 0.3487 - accuracy: 0.8563
Epoch 00100: saving model to checkpoints\weights.100.hdf5
640/854 [=====================>........] - ETA: 0s - loss: 0.3484 - accuracy: 0.8566
Epoch 00100: saving model to checkpoints\weights.100.hdf5

In [ ]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")